In [1]:
import pandas as pd

#### 1) Establish excel writer/workbook- only run once

In [2]:
writer = pd.ExcelWriter("C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/CalEEMod/CalEEMod_LandUseVars_PBA50EIR.xlsx")

#### 2) Loop through remaining cells (informal loop helps debug)

##### a) select file to summarize

In [150]:
# select input file for summarization- will be an EIR run + year (2015 or 2050)
np_2015 = "run314_building_data_2015_NP.csv"
np_2050 = "run314_building_data_2050_NP.csv"
fbp_2015 = "run182_building_data_2015_FBP.csv"
fbp_2050 = "run182_building_data_2050_FBP.csv"
alt1_2015 = "run375_building_data_2015_ALT1.csv"
alt1_2050 = "run375_building_data_2050_ALT1.csv"
alt2_2015 = "run374_building_data_2015_ALT2.csv"
alt2_2050 = "run374_building_data_2050_ALT2.csv"
file = alt2_2050
# name output excel sheet 
sheet = "ALT2 2050"

##### b) continue running

In [151]:
#import building data and subset columns
b = pd.read_csv("C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/CalEEMod/"+file)
b = b[['zone_id', 'building_id', 'parcel_id', 'source', 'building_type', 'residential_units', 'non_residential_sqft']]

In [152]:
# import parcel info and subset columns
p = pd.read_csv("C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/CalEEMod/2021_02_25_parcels_geography.csv")
p = p[['PARCEL_ID', 'ACRES']]

In [153]:
# import TAZ-county crosswalk
xwalk = pd.read_csv("C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/CalEEMod/taz_geography.csv")
xwalk = xwalk[['zone', 'county']]

In [154]:
# replace county numbers with county names
xwalk.loc[xwalk.county == 1, "county"] = "San Francisco"
xwalk.loc[xwalk.county == 2, "county"] = "San Mateo"
xwalk.loc[xwalk.county == 3, "county"] = "Santa Clara"
xwalk.loc[xwalk.county == 4, "county"] = "Alameda"
xwalk.loc[xwalk.county == 5, "county"] = "Contra Costa"
xwalk.loc[xwalk.county == 6, "county"] = "Solano"
xwalk.loc[xwalk.county == 7, "county"] = "Napa"
xwalk.loc[xwalk.county == 8, "county"] = "Sonoma"
xwalk.loc[xwalk.county == 9, "county"] = "Marin"

In [155]:
# join parcel acres to buildings
b = b.merge(p, left_on='parcel_id', right_on='PARCEL_ID', how='left')

In [156]:
# join county to buildings using zone_id
b = b.merge(xwalk, left_on='zone_id', right_on='zone', how='left')

In [157]:
# create DUA column from building and parcel data
b["dua"] = b.residential_units / b.ACRES

In [158]:
# building types
#HS: single family residential
#HT: townhomes
#HM: multi family residential
#OF: office
#HO: hotel
#SC: school
#IL: light industrial
#IW: warehouse
#IH: heavy industrial
#RS: retail general
#RB: retail big box
#MR: mixed use residential focused
#MT: mixed use industiral focused
#ME: mixed use employment focused

In [159]:
# add single family and multifamily building vars
b.loc[(b.building_type == 'HS') | (b.building_type == 'HT'), 'SF'] = b.residential_units
b.loc[(b.building_type == 'HM') | (b.building_type == 'MR') | (b.building_type == 'PA2') | (b.building_type == 'RS') |
      (b.building_type == 'OF') | (b.building_type == 'IL') | (b.building_type == 'HO') | (b.building_type == 'SC'), 
      'MF'] = b.residential_units

In [160]:
# add multifamily low, mid, and high rise building vars
b.loc[(b.MF > 0) & (b.dua < 32), 'MF_low'] = b.residential_units
b.loc[(b.MF > 0) & (b.dua >= 32)& (b.dua < 50), 'MF_mid'] = b.residential_units
b.loc[(b.MF > 0) & (b.dua >= 50), 'MF_high'] = b.residential_units

In [161]:
# add office, retail, industrial, and commercial_other building vars
b.loc[(b.building_type == 'OF'), 'office'] = b.non_residential_sqft
b.loc[(b.building_type == 'ME'), 'mixed_use_emp'] = b.non_residential_sqft
b.loc[(b.building_type == 'RS'), 'retail_general'] = b.non_residential_sqft
b.loc[(b.building_type == 'RB'), 'retail_big_box'] = b.non_residential_sqft
b.loc[(b.building_type == 'MR'), 'mixed_use_res'] = b.non_residential_sqft
b.loc[(b.building_type == 'IL'), 'industrial_light'] = b.non_residential_sqft
b.loc[(b.building_type == 'IH'), 'industrial_heavy'] = b.non_residential_sqft
b.loc[(b.building_type == 'IW'), 'warehouse'] = b.non_residential_sqft
b.loc[(b.building_type == 'MT'), 'mixed_use_industrial'] = b.non_residential_sqft
b.loc[(b.building_type == 'HO'), 'hotel'] = b.non_residential_sqft
b.loc[(b.building_type == 'SC'), 'school'] = b.non_residential_sqft

In [162]:
# debug check- res
debug_res = b[(b.residential_units > 0) & (b.SF.isnull()) & (b.MF.isnull())]
debug_res.building_type.value_counts()

Series([], Name: building_type, dtype: int64)

In [163]:
# debug check- multifamlily 
debug_mf = b[(b.MF > 0) & (b.MF_low.isnull()) & (b.MF_mid.isnull()) & (b.MF_high.isnull())]
debug_mf

,zone_id,building_id,parcel_id,source,building_type,residential_units,non_residential_sqft,PARCEL_ID,ACRES,zone,...,mixed_use_emp,retail_general,retail_big_box,mixed_use_res,industrial_light,industrial_heavy,warehouse,mixed_use_industrial,hotel,school
1812737,353,1843295,2054505,h5_inputs,HM,1655.0,0.0,NaN,NaN,353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
# move building without a PARCEL_ID/ACRES to MF_high manually
b.loc[(b.building_id == 1843295), 'MF_high'] = b.residential_units
# re-check
debug_mf = b[(b.MF > 0) & (b.MF_low.isnull()) & (b.MF_mid.isnull()) & (b.MF_high.isnull())]
debug_mf

,zone_id,building_id,parcel_id,source,building_type,residential_units,non_residential_sqft,PARCEL_ID,ACRES,zone,...,mixed_use_emp,retail_general,retail_big_box,mixed_use_res,industrial_light,industrial_heavy,warehouse,mixed_use_industrial,hotel,school


In [165]:
# debug check- nonres
debug_nonres = b[(b.non_residential_sqft > 0) & (b.office.isnull()) & (b.mixed_use_emp.isnull()) & 
                 (b.retail_general.isnull()) & (b.retail_big_box.isnull()) & (b.mixed_use_res.isnull()) & 
                 (b.industrial_light.isnull()) & (b.industrial_heavy.isnull()) & (b.warehouse.isnull()) & 
                 (b.mixed_use_industrial.isnull()) & (b.hotel.isnull()) & (b.school.isnull())]
print(debug_nonres.building_type.value_counts())
# remaining non res sqft are in HS, HM, HT

HS    18716
HM     4178
HT       12
Name: building_type, dtype: int64


In [166]:
county_summary = b[["SF", 'MF', "MF_low", "MF_mid", "MF_high", "office", "mixed_use_emp", "retail_general", 
                    "retail_big_box", "mixed_use_res", "industrial_light", "industrial_heavy",
                    "warehouse", "mixed_use_industrial", "hotel", "school", "county"]].groupby('county').sum()

In [167]:
county_summary

,SF,MF,MF_low,MF_mid,MF_high,office,mixed_use_emp,retail_general,retail_big_box,mixed_use_res,industrial_light,industrial_heavy,warehouse,mixed_use_industrial,hotel,school
county,,,,,,,,,,,,,,,,
Alameda,371109.0,515367.0,116392.0,84936.0,314039.0,1.941867e+08,1247090.0,65303910.0,6.830582e+06,14660522.0,76246706.0,10073884.0,134346201.0,0.0,11163305.0,21156689.0
Contra Costa,363234.0,205387.0,45392.0,60753.0,99242.0,8.337490e+07,0.0,52026879.0,1.834749e+06,1786589.0,34773719.0,6469603.0,25614776.0,0.0,4571760.0,16917210.0
Marin,86599.0,54092.0,21818.0,19708.0,12566.0,2.867272e+07,0.0,9367341.0,1.596710e+05,1625493.0,0.0,0.0,5199275.0,0.0,834981.0,2893389.0
Napa,45860.0,15642.0,8642.0,3342.0,3658.0,3.213548e+06,0.0,11645532.0,0.000000e+00,235444.0,16074505.0,5050.0,9596063.0,0.0,1752606.0,0.0
San Francisco,112451.0,419371.0,27215.0,34354.0,357802.0,2.008662e+08,2121450.0,28550487.0,2.956105e+06,64130462.0,1643409.0,0.0,32333295.0,0.0,21278428.0,11076191.0
San Mateo,191297.0,213770.0,34355.0,36714.0,142701.0,9.665014e+07,0.0,28777441.0,2.380407e+06,4326616.0,12550747.0,1014535.0,34478532.0,0.0,10518816.0,14440778.0
Santa Clara,428564.0,791521.0,85401.0,99621.0,606499.0,2.379538e+08,9820000.0,115615420.0,1.745055e+07,24971262.0,176542539.0,729796.0,42574761.0,0.0,467922.0,19309848.0
Solano,131235.0,66688.0,25480.0,26089.0,15119.0,2.096367e+07,0.0,27955521.0,3.062300e+04,629870.0,37853122.0,0.0,23255941.0,0.0,2398765.0,722745.0
Sonoma,165209.0,82458.0,41792.0,19958.0,20708.0,6.218853e+07,96278.0,26832611.0,3.424137e+05,1761346.0,1190180.0,5509.0,33651226.0,0.0,4420954.0,972325.0


In [168]:
# run checks
print("compare total res units to SF + MF")
print(b.residential_units.sum())
print(county_summary.SF.sum() + county_summary.MF.sum())
print("compare total MF units to low/mid/high")
print(b.MF.sum())
print(county_summary.MF_low.sum() + county_summary.MF_mid.sum() + county_summary.MF_high.sum())
print("compare total non-res sqft to non-res sqft by category")
print(round(b.non_residential_sqft.sum()))
print(round(county_summary.office.sum() + county_summary.mixed_use_emp.sum() + county_summary.retail_general.sum() + 
            county_summary.retail_big_box.sum() + county_summary.mixed_use_res.sum() + 
            county_summary.industrial_light.sum() + county_summary.industrial_heavy.sum() + 
            county_summary.warehouse.sum() + county_summary.mixed_use_industrial.sum() + county_summary.hotel.sum() + 
            county_summary.school.sum()))

compare total res units to SF + MF
4259854.0
4259854.0
compare total MF units to low/mid/high
2364296.0
2364296.0
compare total non-res sqft to non-res sqft by category
2435636296.0
2314663009.0


In [169]:
# confirm that the non-res sqft discrepancy matches the residential non-res sqft
print(debug_nonres.non_residential_sqft.sum())
print(round(b.non_residential_sqft.sum()) - round(county_summary.office.sum() + county_summary.mixed_use_emp.sum() + 
      county_summary.retail_general.sum() + county_summary.retail_big_box.sum() + county_summary.mixed_use_res.sum() + 
      county_summary.industrial_light.sum() + county_summary.industrial_heavy.sum() + county_summary.warehouse.sum() + 
      county_summary.mixed_use_industrial.sum() + county_summary.hotel.sum() + county_summary.school.sum()))

120973287.0
120973287.0


##### c) add as a sheet to workbook

In [170]:
# write to excel workbook
county_summary.to_excel(writer, sheet_name=sheet)
writer.save()